In [1]:
from keras.models import Model
from keras import layers
from keras import Input
import keras
import numpy as np

weishu=978

yaowu=np.random.randint(1,100,size=(1000,weishu))
babiao=np.random.randint(1,100,size=(1000,weishu))
input1=np.concatenate((yaowu, babiao),axis=1)
#input2=np.stack((yaowu, babiao), axis=1)
output=np.random.randint(0,2,size=(1000))


wide_input=Input(shape=(input1.shape[1],),name='wide')

final_output=layers.Dense(200)(wide_input)
final_output=layers.Dense(10)(final_output)
final_output=layers.Dense(1,activation = 'sigmoid')(final_output)

model = Model(wide_input,final_output)

model.compile(optimizer = 'rmsprop',loss = 'binary_crossentropy',metrics = ['acc'])

model.fit(input1,output,epochs = 10,batch_size = 256)

E:\ProgramData\Anaconda3\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
E:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
E:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)
Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1000/1000 [==============================] - 17s 17ms/step - loss: 7.5138 - acc: 0.5200
Epoch 2/10
1000/1000 [==============================] - 0s 26us/step - loss: 7.9871 - acc: 0.4990
Epoch 3/10
1000/1000 [==============================] - 0s 25us/step - loss: 7.9871 - acc: 0.4990
Epoch 4/10
1000/1000 [==============================] - 0s 24us/step - loss: 7.9871 - acc: 0.4990
Epoch 5/10
1000/1000 [==============================] - 0s 30us/step - loss: 7.9871 - acc: 0.4990
Epoch 6/10
1000/1000 [==============================] - 0s 27us/step - loss: 7.9871 - acc: 0.4990
Epoch 7/10
1000/1000 [==============================] - 0s 24us/step - loss: 7.9871 - acc: 0.4990
Epoch 8/10
1000/1000 [==============================] - 0s 25us/step - loss: 7.9871 - acc: 0.4990
Epoch 9/10
1000/1000 [==============================] - 0s 24us/step - loss: 7.9871 - acc: 0.49

In [ ]:
from sklearn.metrics import accuracy_score ,f1_score,roc_auc_score
import numpy as np
import random
from keras.models import Model
from keras import layers
from keras import Input
from keras import regularizers
import keras

In [ ]:
pos_data=np.load(r'after_merge/A375/pos_data.npy')
neg_data=np.load(r'after_merge/A375/neg_data.npy')

use_neg=random.sample(list(neg_data),2*len(pos_data))
use_neg=np.array(use_neg)

use_data=[]

for d in pos_data:
    use_data.append((d,1))

for d in use_neg:
    use_data.append((d,0))

random.shuffle(use_data)   

x,y=zip(*use_data)

In [ ]:
train_data=np.array(x)
train_targets=np.array(y)
#10折交叉验证
k = 10
num_val_samples = len(train_data) // k
num_epochs = 100
all_acc = []
all_auc = []
all_f1 = []
for i in range(k):
    print('processing fold #', i)
    # 验证集
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # 训练集
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)
    
    #训练数据
    input1=partial_train_data.reshape(partial_train_data.shape[0],partial_train_data.shape[1]*partial_train_data.shape[2])
    input2=partial_train_data
    output=partial_train_targets
    #测试数据
    test_input1=val_data.reshape(val_data.shape[0],val_data.shape[1]*val_data.shape[2])
    test_input2=val_data
    test_output=val_targets
    
    #noredim-3layersdnn
    wide_input=Input(shape=(input1.shape[1],),name='wide')

    final_output=layers.Dense(200)(wide_input)
    final_output=layers.Dense(10)(final_output)
    final_output=layers.Dense(1,activation = 'sigmoid')(final_output)

    model = Model(wide_input,final_output)
    
    model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['acc'])
    
    model.fit(input1,output,epochs = 100,batch_size = 64)
    
    pred=model.predict(test_input1)
    for pred_i in pred:
        pred_i[pred_i >=0.5] = 1
        pred_i[pred_i < 0.5] = 0
    all_f1.append(f1_score(test_output, pred.flatten(), average='micro'))
    all_auc.append(roc_auc_score(test_output, pred.flatten()))
    all_acc.append(accuracy_score(test_output, pred.flatten(),normalize =True))

In [ ]:
def reduce_dim(wide_dim,deep_dim):  
    train_data=np.array(x)
    train_targets=np.array(y)
    #10折交叉验证
    k = 10
    num_val_samples = len(train_data) // k
    num_epochs = 100
    all_acc = []
    all_auc = []
    all_f1 = []
    for i in range(k):
        print('processing fold #', i)
        # 验证集
        val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
        val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

        # 训练集
        partial_train_data = np.concatenate(
            [train_data[:i * num_val_samples],
             train_data[(i + 1) * num_val_samples:]],
            axis=0)
        partial_train_targets = np.concatenate(
            [train_targets[:i * num_val_samples],
             train_targets[(i + 1) * num_val_samples:]],
            axis=0)

        #训练数据
        input1=partial_train_data.reshape(partial_train_data.shape[0],partial_train_data.shape[1]*partial_train_data.shape[2])
        input2=partial_train_data
        output=partial_train_targets
        #测试数据
        test_input1=val_data.reshape(val_data.shape[0],val_data.shape[1]*val_data.shape[2])
        test_input2=val_data
        test_output=val_targets

        #redim-use wad and 3 layers dnn
        wide_input=Input(shape=(input1.shape[1],),name='wide')
        #wide_dim
        wide_output=layers.Dense(wide_dim)(wide_input)

        deep_input=Input(shape=(input2.shape[1],input2.shape[2],),name='deep')
        deep_hide=layers.LSTM(400,return_sequences=True)(deep_input)
        deep_hide=layers.LSTM(400)(deep_hide)
        #deep_dim
        deep_output=layers.Dense(deep_dim)(deep_hide)

        #deep_dim+wide_dim
        concatenated = layers.concatenate([wide_output,deep_output],axis = -1)

        #3dnn
        final_output=layers.Dense(200)(concatenated)
        final_output=layers.Dense(10)(final_output)
        final_output=layers.Dense(1,activation = 'sigmoid')(final_output)

        model = Model([wide_input,deep_input],final_output)

        model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['acc'])

        model.fit([input1,input2],output,epochs = 100,batch_size = 64)

        pred=model.predict(test_input1)
        for pred_i in pred:
            pred_i[pred_i >=0.5] = 1
            pred_i[pred_i < 0.5] = 0
        all_f1.append(f1_score(test_output, pred.flatten(), average='micro'))
        all_auc.append(roc_auc_score(test_output, pred.flatten()))
        all_acc.append(accuracy_score(test_output, pred.flatten(),normalize =True))
        return np.mean(all_acc),np.mean(all_auc),np.mean(all_f1)

wide_dim=[50,100,150,200,250,300]
deep_dim=[100,200,300,400]
all_value=[]
for w in wide_dim:
    for d in deep_dim:
        the_acc,the_auc,the_f1=reduce_dim(w,d)
        all_value.append([w,d,the_acc,the_auc,the_f1])
        print("wide_dim:",w,"deep_dim:",d)
        print("acc:",the_acc,"auc:",the_auc,"f1:",the_f1)
for v in all_value:
    print("wide_dim:",v[0],"deep_dim:",v[1])
    print("acc:",v[2],"auc:",v[3],"f1:",v[4])
    

In [5]:
all_value=[]
a=1
b=2
c=3
all_value.append([a,b,c])

In [6]:
all_value

[[1, 2, 3]]